In [ ]:
import pandas as pd
import datetime
import requests
import json
import sys, os
import itertools
import time
import numpy as np
from os import path
from pathlib import Path
from binance.spot import Spot
from twelvedata import TDClient
from sklearn import metrics
from sklearn.inspection import permutation_importance
from sklearn.model_selection import TimeSeriesSplit
import matplotlib.pyplot as plt
from sklearn.svm import SVC
import itertools
import xgboost as xgb
import seaborn as sns
from typing import Union, Dict, List
import talib
import ta
import shap
sys.path.insert(0, os.path.join(os.getcwd(), ".."))
import fin_utilities 
cfg = fin_utilities.__cfg_reading("pred")
import my_functions

import matplotlib as mpl
# Set the default color cycle
mpl.rcParams.update(mpl.rcParamsDefault)


#td = TDClient(apikey=cfg['TWELVEDATA']['API'])  

PROJECT_DIR = eval(cfg['PROJECT_PATH'])
DATA_PATH   = PROJECT_DIR / cfg['DATA_FOLDER']
SOURCE      = cfg['API_DATA_SOURCE']
URL         = cfg[SOURCE]['API_URL_HIST_DATA']
COLUMNS     = cfg[SOURCE]['COLUMN_NAMES']
PARAMS      = cfg[SOURCE]['REQ_PARAMS']
PAIR        = cfg['SYMBOL'] + cfg['STABLECOIN']
MAX_LENGTH  = cfg['MAX_TRADE_DURATION']
XGB_PARAM   = cfg['xgb']


In [ ]:
if SOURCE == 'BINANCE':
    PARAMS.update({'startTime': my_functions.datetime_to_ts(PARAMS['startTime'])})
    PARAMS.update({'endTime': my_functions.datetime_to_ts(PARAMS['endTime'])})

PARAMS  

limit=1000

In [ ]:
compare_other_symbols = True
# new_connection
client = Spot()

df = my_functions.load_data(client, PAIR, DATA_PATH, PARAMS, COLUMNS)


if compare_other_symbols:

    pairs_available = my_functions.get_all_pairs_available(client)

    # prendo le top coin e rimuovo il simbolo in cfg se c'è
    symbols, actualized_mc = my_functions.get_top_symbols_by_marketcap(50, cfg['STABLECOIN'])
    symbols = [el for el in symbols if el not in PAIR]

    for s in symbols:

        if s in pairs_available:
            
            df = pd.concat([df,my_functions.load_data(client, s, DATA_PATH, PARAMS, COLUMNS)], ignore_index=True)


In [ ]:
df.drop(columns=['OpenTimestamp', 'Ignore.'], inplace=True, errors='ignore')
df['Mkt_Cap'] = df['Close']*df['Volume']

df_mkt_cap = df.groupby(['CloseTimestamp'])['Mkt_Cap'].sum().reset_index()
df_mkt_cap.head(5)

In [ ]:
df_btc = pd.merge(df[df['Symbol']==PAIR], df_mkt_cap, on='CloseTimestamp', suffixes=[None, '_tot'])
df_btc.drop(columns=['Symbol', 'Quote asset volume','Taker buy base asset volume','Taker buy quote asset volume'], inplace=True)
df_btc['CloseTimestamp'] = df_btc['CloseTimestamp'].dt.date
df_btc.head(5)

In [ ]:
df_btc['Norm_wgt_vol'] = df['Volume']*(df_btc['Open']+df_btc['High']+df_btc['Low']+df_btc['Close'])/(4*df_btc['Number of trades'])

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import mplfinance as mpf

# get levels using the first method
levels_1 = my_functions.detect_level_method_1(df_btc)
# get levels using the second method
levels_2 = my_functions.detect_level_method_2(df_btc)



def plot_candlestick_aline(df_, width, height, support,resistance):
    """
    Plot candlestick chart of the dataframe
    """
    dict(hlines=[support,resistance],colors=['g','r'],linestyle='-.')
    df = df_.copy()
    df.rename(columns={df.columns[0]:'Date'}, inplace=True)
    df[df.columns[0]] = pd.to_datetime(df[df.columns[0]])
    df=df.set_index(df.columns[0])
    mpf.plot(df,type='candle', style='charles', figsize=(width, height),alines=dict(alines=support,linestyle='-.'))

def plot_candlestick_hline(df_, width, height, supports,resistance):
    """
    Plot candlestick chart of the dataframe
    """
    df = df_.copy()
    df.rename(columns={df.columns[0]:'Date'}, inplace=True)
    df[df.columns[0]] = pd.to_datetime(df[df.columns[0]])
    df=df.set_index(df.columns[0])
    mpf.plot(df,type='candle', style='charles', figsize=(width, height), hlines=list(supports))

def determine_optimal_clusters(data, max_clusters, plot=False):
    """
    Determines the optimal number of clusters for a given dataset using the elbow method.
    """

    wcss = []
    silhouette_scores = []

    # Loop through a range of possible number of clusters
    for i in range(2, max_clusters+1):
        kmeans = KMeans(n_clusters=i, init='k-means++', n_init=5, random_state=42)
        labels = kmeans.fit_predict(data)
        silhouette_scores.append(silhouette_score(data, labels))
        kmeans.fit(data)
        wcss.append(kmeans.inertia_)


    # Find the "elbow" point on the plot
    kneed_clusters = None
    for i in range(2, len(wcss)):
        if wcss[i] - wcss[i-1] < wcss[i-1] - wcss[i-2]:
            kneed_clusters = i
            break

    optimal_clusters = silhouette_scores.index(max(silhouette_scores)) + 2
    
    if plot:
        # Plot the within-cluster variation for each number of clusters
        plt.plot(range(2, max_clusters+1), wcss)
        plt.title('Elbow Method')
        plt.xlabel('Number of clusters')
        plt.ylabel('Within-cluster variation')
        plt.show()

        plt.plot(range(2, max_clusters+1), silhouette_scores)
        plt.title('Silhouette Method')
        plt.xlabel('Number of clusters')
        plt.ylabel('Silhouette Score')
        plt.show()

    return kneed_clusters, optimal_clusters

    


# determine the optimal number of clusters
kneed, silhouette = determine_optimal_clusters(df_btc[['High','Low','Norm_wgt_vol']],20, False)
print(kneed, silhouette)

# fit KMeans to the data using the optimal number of clusters
kmeans = KMeans(n_clusters=int((kneed+silhouette)/2))
kmeans.fit(df_btc[['High','Low','Norm_wgt_vol']])
pred_y = kmeans.fit_predict(df_btc[['High','Low','Norm_wgt_vol']])

# Extract the cluster centers
cluster_centers = kmeans.cluster_centers_
df_btc['Cluster'] = pred_y

# Use the cluster centers as support and resistance levels
supports = cluster_centers[:, 0]
resistance = cluster_centers[:, 0]

#support = my_functions.list_of_tuple_to_hline(levels_2,df_btc['CloseTimestamp'])
plot_candlestick_hline(df_btc[['CloseTimestamp','Open','High','Low','Close']], 30,20,supports, None)

In [ ]:
df_btc['EMA_VOL_7d'] = my_functions.rolling_kpi(df_btc,'Volume',7, 'ema', False)
df_btc['EMA_VOL_30d'] = df_btc['Volume'].ewm(span=30, adjust=False).mean()
df_btc['EMA_VOL_60d'] = df_btc['Volume'].ewm(span=60, adjust=False).mean()
df_btc['EMA_VOL_100d'] = df_btc['Volume'].ewm(span=100, adjust=False).mean()
df_btc['EMA_VOL_200d'] = df_btc['Volume'].ewm(span=200, adjust=False).mean()
df_btc['STD_VOL_7d'] = my_functions.rolling_kpi(df_btc,'Volume',7, 'std', False)
df_btc['STD_VOL_30d'] = my_functions.rolling_kpi(df_btc,'Volume',30, 'std', False)
df_btc['STD_VOL_60d'] = my_functions.rolling_kpi(df_btc,'Volume',60, 'std', False)
df_btc['STD_VOL_100d'] = my_functions.rolling_kpi(df_btc,'Volume',100, 'std', False)
df_btc['STD_VOL_200d'] = my_functions.rolling_kpi(df_btc,'Volume',200, 'std', False)
df_btc['PCT_Change'] = df_btc['Close'].pct_change()
# Calculate the mean and standard deviation of the daily returns
mean_return = df_btc['PCT_Change'].mean()
std_return = df_btc['PCT_Change'].std()
# Calculate the risk-free rate
risk_free_rate = 0.01
# Calculate the Sharpe ratio
df_btc['Sharpe_Ratio'] = (mean_return - risk_free_rate) / std_return
df_btc['Log_Ret'] = np.log(1 + df_btc['PCT_Change'])
df_btc['Log_Ret_7d'] = np.log(df_btc['PCT_Change']) - np.log(df_btc['PCT_Change'].shift(7))
df_btc['Excursion'] = df_btc['High']-df_btc['Low']
df_btc['PCT_Excursion'] = df_btc['Excursion'].pct_change()
df_btc['AVG_Candle_Price'] = (df_btc['Open']+df_btc['High']+df_btc['Low']+df_btc['Close'])/4
df_btc['AVG_Candle_Price_7d'] = df_btc['AVG_Candle_Price'].ewm(span=7, adjust=False).mean()
df_btc['AVG_Candle_Price_30d'] = df_btc['AVG_Candle_Price'].ewm(span=30, adjust=False).mean()
df_btc['AVG_Candle_Price_60d'] = df_btc['AVG_Candle_Price'].ewm(span=60, adjust=False).mean()
df_btc['AVG_Candle_Price_100d'] = df_btc['AVG_Candle_Price'].ewm(span=100, adjust=False).mean()
df_btc['AVG_Candle_Price_200d'] = df_btc['AVG_Candle_Price'].ewm(span=200, adjust=False).mean()
df_btc['STD_AVG_Candle_Price_7d'] = my_functions.rolling_kpi(df_btc,'AVG_Candle_Price',7, 'std', False)
df_btc['STD_AVG_Candle_Price_30d'] = my_functions.rolling_kpi(df_btc,'AVG_Candle_Price',30, 'std', False)
df_btc['STD_AVG_Candle_Price_60d'] = my_functions.rolling_kpi(df_btc,'AVG_Candle_Price',60, 'std', False)
df_btc['STD_AVG_Candle_Price_100d'] = my_functions.rolling_kpi(df_btc,'AVG_Candle_Price',100, 'std', False)
df_btc['STD_AVG_Candle_Price_200d'] = my_functions.rolling_kpi(df_btc,'AVG_Candle_Price',200, 'std', False)
df_btc['PCT_Mkt_Cap'] = df_btc['Mkt_Cap']/df_btc['Mkt_Cap_tot']
df_btc['DELTA_NTrades'] = df_btc['Number of trades']- df_btc['Number of trades'].shift()
df_btc['EMA_Trades_7d'] = my_functions.rolling_kpi(df_btc,'Number of trades',7, 'mean', False)
df_btc['EMA_Trades_30d'] = my_functions.rolling_kpi(df_btc,'Number of trades',30, 'mean', False)
df_btc['EMA_Trades_60d'] = my_functions.rolling_kpi(df_btc,'Number of trades',60, 'mean', False)
df_btc['EMA_Trades_100d'] = my_functions.rolling_kpi(df_btc,'Number of trades',100, 'mean', False)
df_btc['EMA_Trades_200d'] = my_functions.rolling_kpi(df_btc,'Number of trades',200, 'mean', False)
#Volume-price trend:
df_btc['VP_trend'] = df_btc['Volume'] * df['Close'].pct_change()
#On-balance volume
df_btc['OBV'] = np.where(df_btc['Close'] > df_btc['Close'].shift(1), df_btc['Volume'], - df_btc['Volume'])
df_btc['OBV'] = df_btc['OBV'].cumsum()
#VWAP
df_btc['VWAP'] = (df_btc['Close'] * df_btc['Volume']).cumsum() / df_btc['Volume'].cumsum()
#Volume Rate Of Change
df_btc['Volume_ROC'] = df_btc['Volume'].pct_change()
#Volume-price trend divergence:
df_btc['VP_divergence'] = df_btc['Volume'].rolling(window=10).mean() - df_btc['Close'].rolling(window=10).mean()
#Chaikin Money Flow
df_btc["chaikin_money_flow"] = ((df_btc['Close'] - df_btc['Low']) - (df_btc['High'] - df_btc['Close'])) / (df_btc['High'] - df_btc['Low']) * df_btc['Volume']
# Head and Shoulders
#df_btc['hs_pattern'] = ta.pattern.head_and_shoulders(df['Close'],neck_width=1,trend='down')
# 2 crows or 3 black crows candlestick pattern 
#df_btc['cdl2crows'] = talib.CDL2CROWS(df_btc['Open'],df_btc['High'],df_btc['Low'],df_btc['Close'])
#df_btc['cdl3blackcrows'] = talib.CDL3BLACKCROWS(df_btc['Open'],df_btc['High'],df_btc['Low'],df_btc['Close'])
# RSI
df_btc['RSI'] = ta.momentum.RSIIndicator(df_btc['Close'], window=14).rsi()
#BollingerBands
bb = ta.volatility.BollingerBands(df_btc['Close'], window=20, window_dev=2)
df_btc['BB_upper'] = bb.bollinger_hband()
df_btc['BB_lower'] = bb.bollinger_lband()
#MFI
mfi = ta.volume.MFIIndicator(df_btc['High'], df_btc['Low'], df_btc['Close'], df_btc['Volume'], window=14)
df_btc['MFI'] = mfi.money_flow_index()

In [ ]:
#df_btc = labelize_output_according_criterion(df_btc, ['Open','High','Low','Close'], max_trade_length=MAX_LENGTH)
df_btc = my_functions.labelize_output_according_criterion2(df_btc, ['Open','High','Low','Close'], max_trade_length=MAX_LENGTH)
df_btc['signal'] = (df_btc['min_above'].lt(df_btc['min_below'])) & (df_btc['min_above'] <= MAX_LENGTH)
df_btc['signal']=df_btc['signal'].astype(int)
df_btc.tail(10)


In [ ]:
df_btc.to_csv('test.csv', index=False)

In [ ]:
dt_threshold = df_btc.iloc[-100]['CloseTimestamp']

df_btc.drop(columns=['CloseAbovethreshold', 'HighAbovethreshold',
       'LowAbovethreshold', 'OpenAbovethreshold', 'CloseBelowthreshold',
       'HighBelowthreshold', 'LowBelowthreshold', 'OpenBelowthreshold', 'TP',
       'SL', 'min_above', 'min_below'], inplace=True, errors='ignore')
train, test = my_functions.split_train_validation(df_btc,'CloseTimestamp', dt_threshold )
train.drop(columns=['CloseTimestamp'], inplace=True)
test.drop(columns=['CloseTimestamp'], inplace=True)

In [ ]:
# get levels using the first method
levels_1 = my_functions.detect_level_method_1(df_btc)
# get levels using the second method
levels_2 = my_functions.detect_level_method_2(df_btc)

import mplfinance as mpf


def plot_candlestick(df_, width, height, support,resistance):
    """
    Plot candlestick chart of the dataframe
    """
    dict(hlines=[support,resistance],colors=['g','r'],linestyle='-.')
    df = df_.copy()
    df.rename(columns={df.columns[0]:'Date'}, inplace=True)
    df[df.columns[0]] = pd.to_datetime(df[df.columns[0]])
    df=df.set_index(df.columns[0])
    mpf.plot(df,type='candle', style='charles',  figsize=(width, height),alines=dict(alines=support,linestyle='-.'))


support = my_functions.list_of_tuple_to_hline(levels_2,df_btc['CloseTimestamp'])

plot_candlestick(df_btc[['CloseTimestamp','Open','High','Low','Close']], 30,20,{}, np.array(levels_2)[:,1])



In [ ]:
"""tscv = TimeSeriesSplit(gap=0, max_train_size=100, n_splits=60, test_size=30)
p = {  
    'eval_metric':'auc',
    'tree_method':'hist',
    'lambda':5,
    'max_depth':3,
    'scale_pos_weight':3,
    'objective':'binary:logistic',
    'eta':0.01,
    'n_estimators':2000
    }

model = my_functions.model_selection('xgb',p)

for train_index, val_index in tscv.split(train):
    print(f'Train index:{train_index[0]}:{train_index[-1]}, Val index:{val_index[0]}:{val_index[-1]}')
    train_features, train_labels    = my_functions.split_target_features(train.loc[train_index],cfg['TARGET_VARIABLE'])
    val_features, val_labels        = my_functions.split_target_features(train.loc[val_index],cfg['TARGET_VARIABLE'])
    model.fit(train_features, train_labels)
    print("Test set score: {:.2f}".format(model.score(val_features, val_labels)))
"""

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

# load your data
for train_index, test_index in tscv.split(train):
    print(f'Train index:{train_index[0]}:{train_index[-1]}, Val index:{test_index[0]}:{test_index[-1]}')
    train_features, train_labels    = my_functions.split_target_features(train.loc[train_index],cfg['TARGET_VARIABLE'])
    val_features, val_labels        = my_functions.split_target_features(train.loc[test_index],cfg['TARGET_VARIABLE'])
    X_train, X_test = train_features, val_features
    y_train, y_test = train_labels, val_labels

    # create the ensemble model
    clf2 = DecisionTreeClassifier()
    clf3 = xgb.XGBClassifier()

    ensemble_clf = VotingClassifier(estimators=[('dt', clf2), ('svc', clf3)], voting='hard')

    # train the ensemble model
    ensemble_clf.fit(X_train.fillna(-1), y_train)

In [ ]:
ensemble_clf.score(X_test.fillna(-1),y_test)

In [ ]:
X_train

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.layers import Dropout, Activation
from sklearn.preprocessing import MinMaxScaler
from keras.layers import TimeDistributed


X_train, y_train = my_functions.split_target_features(train[:-100],cfg['TARGET_VARIABLE'])
X_val, y_val = my_functions.split_target_features(train[-100:],cfg['TARGET_VARIABLE'])
#
#scaler = MinMaxScaler(feature_range=(0, 1))
#X_train = scaler.fit_transform(X_train.fillna(-1))
#X_val = scaler.transform(X_val.fillna(-1))

# Create the model
X_train = X_train.fillna(-1)

model = Sequential()
model.add(LSTM(100, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(100, return_sequences=False))
model.add(Dense(50))
model.add(Dense(1, activation='sigmoid'))
model.summary()

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam')

# Fit the model on the training data
model.fit(X_train, y_train, batch_size= 5, epochs=3)

In [ ]:
X_test, y_test = my_functions.split_target_features(test,cfg['TARGET_VARIABLE'])
#X_test_scaled = scaler.transform(X_test.fillna(-1))
y_pred = model.predict(X_test.fillna(-1))

# Print the predictions
#print(y_pred)

from sklearn.metrics import roc_auc_score

# Calculate the AUC
auc = roc_auc_score(y_test, y_pred)

# Print the AUC
print("AUC: ", auc)

In [ ]:
y_pred

In [ ]:
from sklearn.metrics import confusion_matrix

def cm_analysis(
    y_true, 
    y_pred, 
    title: str,
    filename: str, 
    labels: List[str], 
    ymap=None, 
    figsize=(10,10)
) -> None:
    """
    Generate matrix plot of confusion matrix with pretty annotations.
    The plot image is saved to disk.

    args: 
      y_true:    true label of the data, with shape (nsamples,)
      y_pred:    prediction of the data, with shape (nsamples,)
      title:     plot name
      filename:  filename of figure file to save
      labels:    string array, name the order of class labels in the confusion matrix.
                 use `clf.classes_` if using scikit-learn models.
                 with shape (nclass,).
      ymap:      dict: any -> string, length == nclass.
                 if not None, map the labels & ys to more understandable strings.
                 Caution: original y_true, y_pred and labels must align.
      figsize:   the size of the figure plotted.

    Returns:
        None
    """

    if ymap is not None:
        y_pred = [ymap[yi] for yi in y_pred]
        y_true = [ymap[yi] for yi in y_true]
        labels = [ymap[yi] for yi in labels]
    cm = confusion_matrix(y_true, y_pred, labels=labels)
    cm_sum = np.sum(cm, axis=1, keepdims=True)
    cm_perc = cm / cm_sum.astype(float) * 100
    annot = np.empty_like(cm).astype(str)
    nrows, ncols = cm.shape
    for i in range(nrows):
        for j in range(ncols):
            c = cm[i, j]
            p = cm_perc[i, j]
            if i == j:
                s = cm_sum[i]
                annot[i, j] = '%.2f%%\n%d/%d' % (p, c, s)
            elif c == 0:
                annot[i, j] = ''
            else:
                annot[i, j] = '%.2f%%\n%d' % (p, c)
    cm = pd.DataFrame(cm, index=labels, columns=labels)
    cm.index.name = 'Actual'
    cm.columns.name = 'Predicted'
    cm_perc = pd.DataFrame(cm_perc, index=labels, columns=labels)
    cm_perc.index.name = 'Actual'
    cm_perc.columns.name = 'Predicted'
    fig, ax = plt.subplots(figsize=figsize)
    sns.heatmap(cm_perc, annot=annot, fmt='', linewidths=1, ax=ax)
    plt.title('\n'+title+'\n', fontsize=14)
    plt.savefig(filename)


y_hat = [1 if i>.5 else 0 for i in y_pred]
cm_analysis(y_test, y_hat,title='asdasda',filename='cm.png',labels=[1,0])

In [ ]:
def model_selection(
    model_type: str,
    parameter: Dict
) -> xgb.XGBClassifier:
    """
    Instantiate a tree based model
    
     Args:
        model_type: xgb/lgb currently supported
        parameter:  default parameters 
    
    Returns:
        Union[xgb.XGBClassifier, LGBMClassifier]: model instantiated
    """

    dict_model = {
            'xgb': xgb.XGBClassifier(n_jobs=-1, random_state=42),
            'rf': RandomForestClassifier(n_jobs=-1, random_state=42),
        }

    return dict_model[model_type].set_params(**parameter)


tscv = TimeSeriesSplit(n_splits=15, test_size=100)
p = {  
    'eval_metric':'auc',
    'tree_method':'hist',
    'lambda':5,
    'gamma':2,
    'max_depth':7,
    'scale_pos_weight':2,
    'objective':'binary:logistic',
    #'subsample': .95,
    'colsample_bytree': .9,
    'min_child_weight':1,
    'eta':0.05,
    'n_estimators':2000
}

model = model_selection('xgb',p)


for train_index, test_index in tscv.split(train):
    print(f'Train index:{train_index[0]}:{train_index[-1]}, Val index:{test_index[0]}:{test_index[-1]}')
    train_features, train_labels    = my_functions.split_target_features(train.loc[train_index],cfg['TARGET_VARIABLE'])
    val_features, val_labels        = my_functions.split_target_features(train.loc[test_index],cfg['TARGET_VARIABLE'])
    X_train, X_test = train_features, val_features
    y_train, y_test = train_labels, val_labels

    model.fit(X_train, y_train, eval_set = [(X_train, y_train), (X_test, y_test)], verbose=500)
    

In [ ]:
y_hat = model.predict_proba(train[train.columns[:-1]])[:,1]
y = train['signal']
df_test= pd.DataFrame({'y':y,'y_hat':y_hat})
df_test = pd.merge(train, df_test, left_index=True, right_index=True)
df_test = df_test[df_test.columns[1:]]
df_test.sort_values(by=['y_hat'], ascending=False, inplace=True)
df_test

In [ ]:
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(train[train.columns[:-1]])
shap.summary_plot(shap_values, train[train.columns[:-1]], plot_size=[30,30], max_display=len(train[train.columns[:-1]]))

In [ ]:
"""from sklearn.model_selection import GridSearchCV

#train.drop(columns=['Open',
# 'EMA_Trades_100d',
# 'EMA_Trades_7d',
# 'AVG_Candle_Price',
# 'STD_VOL_100d',
# 'STD_VOL_30d',
# 'EMA_VOL_200d',
# 'Log_Ret',
# 'High',
# 'Low'], inplace=True)
tscv = TimeSeriesSplit(gap=0, max_train_size=100, n_splits=36, test_size=30)
p = {  
    'eval_metric' : ['auc'],
    'tree_method': ['hist'],
    'lambda':[5],
    'max_depth':[3],
    'scale_pos_weight':[3],
    'objective':['binary:logistic'],
    'eta':[0.01],
    'early_stopping_rounds':[100],
    'n_estimators': [1000, 2000]
    }


custom_cv = []

for i in tscv.split(train):
    custom_cv.append((i[0],i[1]))

model = my_functions.model_selection('xgb',{'early_stopping_rounds':100})
#model.set_params(n_estimators=2000)
gridSearch = GridSearchCV(estimator=model, param_grid=p, cv=custom_cv)
train_features, train_labels    = my_functions.split_target_features(train,cfg['TARGET_VARIABLE'])
gridSearch.fit(train_features, # shoud have shape of (n_samples, n_features) 
                train_labels) #this should be an array with shape (n_samples,)
print(gridSearch.best_score_, gridSearch.best_params_)

for i, (train_index, val_index) in enumerate(tscv.split(train)):
    print(f'iteration: {i}, Train index:{train_index[0]}:{train_index[-1]}, Val index:{val_index[0]}:{val_index[-1]}')
    train_features, train_labels    = my_functions.split_target_features(train.loc[train_index],cfg['TARGET_VARIABLE'])
    val_features, val_labels        = my_functions.split_target_features(train.loc[val_index],cfg['TARGET_VARIABLE'])
    model_ = my_functions.dummy_train(
                    train_features, 
                    train_labels, 
                    val_features, 
                    val_labels, 
                    {  
                    'eval_metric' : 'auc',
                    'tree_method': 'hist',
                    'lambda':5,
                    'max_depth':3,
                    'scale_pos_weight':10,
                    'objective':'binary:logistic',
                    'eta':0.005,
                    }, 
                    2000,
                    'xgb',
                    verbosity=None
            )"""

In [ ]:
from xgboost import plot_importance

fig, ax = plt.subplots(figsize=(12,20)) 
plot_importance(model, ax=ax, importance_type='gain')
plt.show()

In [ ]:
#my_functions.plt_correlation(train)

In [ ]:
'''test.drop(columns=['Open',
 'EMA_Trades_100d',
 'EMA_Trades_7d',
 'AVG_Candle_Price',
 'STD_VOL_100d',
 'STD_VOL_30d',
 'EMA_VOL_200d',
 'Log_Ret',
 'High',
 'Low'], inplace=True)'''
y_hat = clf.predict_proba(test.drop(columns='signal'))[:,1]
y_hat

In [ ]:
import sklearn
import scikitplot as skplt
import matplotlib.pyplot as plt

fpr_test, tpr_test, _ = sklearn.metrics.roc_curve(test['signal'], y_hat)
auc_test = sklearn.metrics.auc(fpr_test, tpr_test)
print(auc_test)
y_true = test['signal'].to_list()
y_probas = y_hat.tolist()
skplt.metrics.plot_roc_curve(test['signal'],clf.predict_proba(test.drop(columns='signal').fillna(-1)), title=f"ROC curve, AUC=test: {auc_test:.4f}", curves=('macro',1), figsize=(10,10))
plt.show()

In [ ]:
test